<div >
<img src = "../banner.jpg" />
</div>

<a target="_blank" href="https://colab.research.google.com/github/ignaciomsarmiento/BDML_202402/blob/main/Lecture01/L01_01_LR.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Lógica basica de la predicción con regresión lineal

Nuestro objetivo  es construir un modelo predictivo de salario

$$
w = f(X) + u
$$

donde w es el salario y X es una matriz que incluye posibles variables/predictores explicativas (ej. educación, edad, educación de los padres, habilidad cognitiva, etc.). 

Un ejemplo de modelo predictivo es la regresión lineal con una forma funcional específica:

$$
ln(w)  = \beta_0 + \beta_1 X_1 + \dots + \beta_p X_p + u
$$

donde $ln(w)$ es el logaritmo del salario.

Para ilustrar, usaré una muestra del NLSY97. El NLSY97 es una muestra representativa a nivel nacional de 8.984 hombres y mujeres nacidos entre 1980 y 1984 y que vivían en los Estados Unidos durante la encuesta inicial de 1997. Los participantes tenían entre 12 y 16 años al 31 de diciembre de 1996. Las entrevistas se realizaron anualmente desde 1997 hasta 2011 y cada dos años desde entonces.

Carguemos los paquetes.

In [ ]:
# install.packages("pacman") #Correr esta linea en Google Colab

In [ ]:
# Cargamos los paquetes

require("pacman")

p_load("tidyverse","stargazer")


Cargamos la base:

In [ ]:
nlsy <- read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/datasets/main/nlsy97.csv')

#Housekeeping
nlsy <- nlsy  %>%   drop_na(educ) #dropea los valores faltantes (NA)

In [ ]:
head(nlsy)

## Regresión Lineal

In [ ]:
reg1 <- lm(lnw_2016~educ,   data = nlsy)

In [ ]:
reg2 <- lm(lnw_2016~educ+ exp,   data = nlsy)

In [ ]:
stargazer(reg1,reg2,type="text") #Similar en stata: outreg2 

## Predicción

$$
\hat{y} = \hat{\beta}_0 +  \hat{\beta}_1 educ 
$$

In [ ]:
reg1$coef

In [ ]:
reg1$coef[1]

In [ ]:
reg1$coef[1]+reg1$coef[2]*16

In [ ]:
exp(reg1$coef[1]+reg1$coef[2]*16)

## Mejor predictor

Vamos a constuir regresores polinomiales de forma tal de tnere reglas de predicción más flexibles y potencialmente mejores.

In [ ]:
#Polinomios de educación

nlsy <- nlsy  %>% mutate(educ2=educ^2,
                         educ3=educ^3,
                         educ4=educ^4,
                         educ5=educ^5,
                         educ6=educ^6,
                         educ7=educ^7,
                         educ8=educ^8
                        )

In [ ]:
head(nlsy)

 Corremos la regresión

In [ ]:
reg3<- lm(lnw_2016~educ+ educ2 + educ3 + educ4 + educ5 + educ6 + educ7 + 
    educ8 ,   data = nlsy)


In [ ]:
stargazer(reg1,reg2,reg3,type="text") #Similar en stata: outreg2 

In [ ]:
#predict me simplifica hacer la operación X\hat{\beta}

nlsy <- nlsy  %>% mutate(yhat=predict(reg3)) #con la función mutate



In [ ]:
head(nlsy)

In [ ]:
# Voy a calcular E(y|X=x) y E(\hat(y)| X=x)

summ = nlsy %>%  
  group_by(
    educ, educ2, educ3, educ4, educ5, 
    educ6, educ7, educ8
  ) %>%  
  summarize(
    mean_y = mean(lnw_2016),
    yhat_reg = mean(yhat), .groups="drop"
  ) 

In [ ]:
head(summ)

In [ ]:
# graficamos
ggplot(summ) + 
  geom_point(
    aes(x = educ, y = mean_y),
    color = "blue", size = 2
  ) + 
  geom_line(
    aes(x = educ, y = yhat_reg), 
    color = "green", linewidth = 1.5
  ) + 
  labs(
    title = "ln Wages by Education in the NLSY",
    x = "Years of Schooling",
    y = "ln Wages"
  ) +
  theme_bw()


Como podemos ver, est modelo flexible captura muy bien patrones complejos. 

Incluyendo un conjunto lo suficientemente rico de transformaciones, las predicciones de OLS producirán estimaciones insesgadas del verdadero predictor ideal: la función de expectativa condicional.